In [1]:
# Import packages and functions
import numpy as np
from scipy.stats import rankdata
from scipy.stats import binom
from itertools import product
from tqdm import tqdm

# Ties and zeros in Wilcoxon signed rank test

In [2]:
# Define type I error level
alpha = 0.05

# Define m0
m0 = 3.7

In [3]:
# Our sample
lens = np.array([3.7, 6.1, 3.1] + [5.0, 3.9, 5.2, 5.5, 2.8, 6.1, 6.4, 2.6, 1.7, 4.3])

print("Absolute values of Xi - m0", np.abs(lens - m0))
n_all = len(lens)
print("Size of the sample", n_all)
n_no_zeros = n_all - np.sum(lens == m0)
print("Size of the sample without zeros", n_no_zeros)

Absolute values of Xi - m0 [0.  2.4 0.6 1.3 0.2 1.5 1.8 0.9 2.4 2.7 1.1 2.  0.6]
Size of the sample 13
Size of the sample without zeros 12


In [4]:
# Compute all ranks
ranks_pratt = rankdata(np.abs(lens - m0), method='ordinal')[np.abs(lens - m0) > 0]
ranks_wilcoxon = rankdata(np.abs(lens - m0)[np.abs(lens - m0) > 0], method='ordinal')
# ORDINAL: All values are given a distinct rank, corresponding to the order that the values occur in the sample - tiebreaking rule

print("Ranks of the sample:")
print("- Pratt's ranks", ranks_pratt)
print("- Wilcoxon's ranks", ranks_wilcoxon)

Ranks of the sample:
- Pratt's ranks [11  4  7  2  8  9  5 12 13  6 10  3]
- Wilcoxon's ranks [10  3  6  1  7  8  4 11 12  5  9  2]


In [5]:
print("Lens above m0", lens[lens > m0])

Lens above m0 [6.1 5.  3.9 5.2 5.5 6.1 6.4 4.3]


In [6]:
# Removing the entry corresponding to zero
mask = (lens > m0)[np.abs(lens - m0) > 0]

# Pick positive ranks
print("Positive ranks:")
print("- Pratt's ranks", ranks_pratt[mask]) 
print("- Wilcoxon's ranks", ranks_wilcoxon[mask], "\n")

# Pick negative ranks
print("Negative ranks:")
print("- Pratt's ranks", ranks_pratt[~mask]) 
print("- Wilcoxon's ranks", ranks_wilcoxon[~mask])

Positive ranks:
- Pratt's ranks [11  7  2  8  9 12 13  3]
- Wilcoxon's ranks [10  6  1  7  8 11 12  2] 

Negative ranks:
- Pratt's ranks [ 4  5  6 10]
- Wilcoxon's ranks [3 4 5 9]


In [7]:
# Wilcoxon signed rank test
W_pos_pratt = np.sum(ranks_pratt[mask])
W_neg_pratt = np.sum(ranks_pratt[~mask])

W_pos_wilcoxon = np.sum(ranks_wilcoxon[mask])
W_neg_wilcoxon = np.sum(ranks_wilcoxon[~mask])

print("Wilcoxon sign rank test statistic:")
print("- with Pratt's ranks", W_pos_pratt)
print("- with Wilcoxon's ranks", W_pos_wilcoxon, "\n")

print("Sum of negative ranks:")
print("- with Pratt's ranks", W_neg_pratt)
print("- with Wilcoxon's ranks", W_neg_wilcoxon)

Wilcoxon sign rank test statistic:
- with Pratt's ranks 65
- with Wilcoxon's ranks 57 

Sum of negative ranks:
- with Pratt's ranks 25
- with Wilcoxon's ranks 21


![caption](./Table.png)

In [8]:
crit_val_pratt = 17
crit_val_wilcoxon = 13

print("Test H0 with Pratt's ranks:")
if min(W_pos_pratt, W_neg_pratt) < crit_val_pratt:
    print("Reject H0\n")
else:
    print("Do not reject H0\n")

print("Test H0 with Wilcoxon's ranks:")
if min(W_pos_wilcoxon, W_neg_wilcoxon) < crit_val_wilcoxon:
    print("Reject H0")
else:
    print("Do not reject H0")

Test H0 with Pratt's ranks:
Do not reject H0

Test H0 with Wilcoxon's ranks:
Do not reject H0


# Point estimation and Confidence Intervals for Median

In [9]:
# Our sample
lens_no_zeros_ties = np.array([5.0, 3.9, 5.2, 5.5, 2.8, 6.1, 6.4, 2.6, 1.7, 4.3])

n = len(lens_no_zeros_ties)
print("Size of the sample", n)

Size of the sample 10


In [10]:
# Calculate Walsh averages
WA = []

for i, xi in enumerate(lens_no_zeros_ties):
    for xj in lens_no_zeros_ties[i:]:
        WA += [(xi + xj) / 2]

WA = np.array(WA)
print("Walsh averages", WA)
print("Size of Walsh averages set", len(WA))

Walsh averages [5.   4.45 5.1  5.25 3.9  5.55 5.7  3.8  3.35 4.65 3.9  4.55 4.7  3.35
 5.   5.15 3.25 2.8  4.1  5.2  5.35 4.   5.65 5.8  3.9  3.45 4.75 5.5
 4.15 5.8  5.95 4.05 3.6  4.9  2.8  4.45 4.6  2.7  2.25 3.55 6.1  6.25
 4.35 3.9  5.2  6.4  4.5  4.05 5.35 2.6  2.15 3.45 1.7  3.   4.3 ]
Size of Walsh averages set 55


In [11]:
# Compute point estimations
m_sample = np.median(lens_no_zeros_ties)
m_hl = np.median(WA)

print("Median point estimation:")
print("- Sample median", m_sample)
print("- Hodges-Lehmann estimator", m_hl)

Median point estimation:
- Sample median 4.65
- Hodges-Lehmann estimator 4.449999999999999


In [12]:
# Find exact distribution of W
def distr_W(n):
    prob = {}

    # generate binary sets representing + and - sign of the ranks
    bin_arr = [np.array(p) for p in product([1, 0], repeat=n)]

    # check all possible values of W
    for pos_neg in tqdm(bin_arr):
        W = np.sum(np.arange(1, n + 1) * pos_neg)
        if W in prob:
            prob[W] += 1
        else:
            prob[W] = 1

    # compute probabilities
    for W in prob:
        prob[W] /= 2 ** n

    return prob

W_prob = distr_W(n) # Compute pmf
W_prob = {k: v for k, v in sorted(W_prob.items(), key=lambda item: item[0])} # Sort the values of W in ascending order
W_cdf = dict(zip(W_prob.keys(), np.cumsum(list(W_prob.values())))) # Compute cdf

100%|██████████| 1024/1024 [00:00<00:00, 214984.85it/s]


In [13]:
# Define confidence level
gamma = 0.999
alpha = 1 - gamma

In [14]:
# Compute confidence intervals
k_a_sample = int(binom(n=n, p=0.5).ppf(alpha / 2)) + 1 # First, compute a quantile of Binom: ppf will return k such that P(Binom \leq k), so, we add + 1
left_sample = np.sort(lens_no_zeros_ties)[k_a_sample - 1] # Then, pick corresponding order statistics: ordering starts with 0, so we add - 1
right_sample = np.sort(lens_no_zeros_ties)[n - k_a_sample]

k_a_hl = list(W_cdf.keys())[np.argmin(np.abs(np.array(list(W_cdf.values())) - alpha / 2))] + 1 # First, compute a quantile of the exact distribution of W
left_hl = np.sort(WA)[k_a_hl - 1] # Then, pick corresponding order statistics
right_hl = np.sort(WA)[n * (n + 1) // 2 - k_a_hl]

print("Median confidence interval:")
print("- with Sign test quantiles [", left_sample, right_sample, "]")
print("- with Wilcoxon signed rank test quantiles [", left_hl, right_hl, "]")

Median confidence interval:
- with Sign test quantiles [ 1.7 6.4 ]
- with Wilcoxon signed rank test quantiles [ 1.7 6.4 ]


In [15]:
# Define another confidence level
gamma = 0.99
alpha = 1 - gamma

In [16]:
# Compute confidence intervals
k_a_sample = int(binom(n=n, p=0.5).ppf(alpha / 2)) + 1 # First, compute a quantile of Binom: ppf will return k such that P(Binom \leq k), so, we add + 1
left_sample = np.sort(lens_no_zeros_ties)[k_a_sample - 1] # Then, pick corresponding order statistics: ordering starts with 0, so we add - 1
right_sample = np.sort(lens_no_zeros_ties)[n - k_a_sample]

k_a_hl = list(W_cdf.keys())[np.argmin(np.abs(np.array(list(W_cdf.values())) - alpha / 2))] + 1 # First, compute a quantile of the exact distribution of W
left_hl = np.sort(WA)[k_a_hl - 1] # Then, pick corresponding order statistics
right_hl = np.sort(WA)[n * (n + 1) // 2 - k_a_hl]

print("Median confidence interval:")
print("- with Sign test quantiles [", left_sample, right_sample, "]")
print("- with Wilcoxon signed rank test quantiles [", left_hl, right_hl, "]")

Median confidence interval:
- with Sign test quantiles [ 2.6 6.1 ]
- with Wilcoxon signed rank test quantiles [ 2.6 5.95 ]


In [17]:
# Define another confidence level
gamma = 0.95
alpha = 1 - gamma

In [18]:
# Compute confidence intervals
k_a_sample = int(binom(n=n, p=0.5).ppf(alpha / 2)) + 1 # First, compute a quantile of Binom: ppf will return k such that P(Binom \leq k), so, we add + 1
left_sample = np.sort(lens_no_zeros_ties)[k_a_sample - 1] # Then, pick corresponding order statistics: ordering starts with 0, so we add - 1
right_sample = np.sort(lens_no_zeros_ties)[n - k_a_sample]

k_a_hl = list(W_cdf.keys())[np.argmin(np.abs(np.array(list(W_cdf.values())) - alpha / 2))] + 1 # First, compute a quantile of the exact distribution of W
left_hl = np.sort(WA)[k_a_hl - 1] # Then, pick corresponding order statistics
right_hl = np.sort(WA)[n * (n + 1) // 2 - k_a_hl]

print("Median confidence interval:")
print("- with Sign test quantiles [", left_sample, right_sample, "]")
print("- with Wilcoxon signed rank test quantiles [", left_hl, right_hl, "]")

Median confidence interval:
- with Sign test quantiles [ 2.8 5.5 ]
- with Wilcoxon signed rank test quantiles [ 3.25 5.55 ]


In [19]:
# Define one more confidence level
gamma = 0.8 # Unlike 95% and 99%, 80%-confidence level CIs are not common. This one is given for comparison purposes only
alpha = 1 - gamma

In [20]:
# Compute confidence intervals
k_a_sample = int(binom(n=n, p=0.5).ppf(alpha / 2)) + 1 # First, compute a quantile of Binom: ppf will return k such that P(Binom \leq k), so, we add + 1
left_sample = np.sort(lens_no_zeros_ties)[k_a_sample - 1] # Then, pick corresponding order statistics: ordering starts with 0, so we add - 1
right_sample = np.sort(lens_no_zeros_ties)[n - k_a_sample]

k_a_hl = int(binom(n=n * (n + 1) // 2, p=0.5).ppf(alpha / 2)) + 1 # First, compute a quantile of Binom
left_hl = np.sort(WA)[k_a_hl - 1] # Then, pick corresponding order statistics
right_hl = np.sort(WA)[n - k_a_hl]

print("Median confidence interval:")
print("- with Sign test quantiles [", left_sample, right_sample, "]")
print("- with Wilcoxon signed rank test quantiles [", left_hl, right_hl, "]")

Median confidence interval:
- with Sign test quantiles [ 3.9 5.2 ]
- with Wilcoxon signed rank test quantiles [ 4.1 5.2 ]
